In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/dataset.zip /content/
!unzip -q dataset.zip -d dataset


In [ ]:
!git clone https://github.com/BioMikeUkr/dl-first-task.git

Cloning into 'dl-first-task'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 35 (delta 14), reused 26 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 76.34 KiB | 15.27 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
import json
import random
random.seed(42)

data = json.load(open("data.json", "rt"))
random.shuffle(data)
print(len(data))
train_data = data[:int(len(data)*0.95)]
print(len(data))
test_data = data[int(len(data)*0.95):]


3047
3047


In [ ]:
from transformers import Trainer, TrainingArguments
from src.model import ClipSegMultiClassModel
from src.config import ClipSegMultiClassConfig
from src.data_processing import SingleClassSegmentationDataset, SegmentationCollator

class_labels = ["background", "Pig", "Horse", "Sheep"]

config = ClipSegMultiClassConfig(
    class_labels=class_labels,
    label2color={
        0: [0, 0, 0],
        1: [255, 0, 0],
        2: [0, 255, 0],
        3: [0, 0, 255]
    },
    model="CIDAS/clipseg-rd64-refined"
)

model = ClipSegMultiClassModel(config).to("cuda")

train_dataset = SingleClassSegmentationDataset(
    dataset=train_data,
    class_labels=class_labels,
    image_size=352
)

collator = SegmentationCollator(model.processor, class_labels)

training_args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=64,
    learning_rate=1e-5,
    num_train_epochs=4,
    logging_dir="./logs",
    logging_steps=5,
    save_strategy="epoch",
    evaluation_strategy="no",
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=None,
    data_collator=collator,
)

trainer.train()

model.save_pretrained("./trained_clipseg_multiclass")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-ebb91b0a379e>:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding', 'truncation'
  return self.preprocess(images, **kwargs)


Step,Training Loss
5,1.917000
10,1.752800
15,1.734600
20,1.713700
25,1.664300
30,1.667200
35,1.605100
40,1.596300
45,1.551700
50,1.581200


/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding', 'truncation'
  return self.preprocess(images, **kwargs)
/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding', 'truncation'
  return self.preprocess(images, **kwargs)
/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding', 'truncation'
  return self.preprocess(images, **kwargs)


In [ ]:
from torch.utils.data import DataLoader

val_dataset = SingleClassSegmentationDataset(test_data, class_labels)
val_collator = SegmentationCollator(model.processor, class_labels)
val_loader = DataLoader(val_dataset, batch_size=4, collate_fn=val_collator)

metrics = model.evaluate(val_loader)
print(metrics)



/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding', 'truncation'
  return self.preprocess(images, **kwargs)


{'accuracy': 0.676349586386167, 'precision': 0.7460084334713697, 'recall': 0.5035276059626524, 'f1': 0.6009564417856172}


In [ ]:
from PIL import Image
import torch


model.eval()
model = ClipSegMultiClassModel.from_pretrained("trained_clipseg_multiclass").to("cuda")
image = Image.open("pigs.jpg").convert("RGB")
mask = model.predict(image)
print(mask.shape)

import numpy as np
import matplotlib.pyplot as plt

def visualize_mask(mask_tensor: torch.Tensor, label2color: dict):

    if mask_tensor.dim() == 3:
        mask_tensor = mask_tensor.squeeze(0)

    mask_np = mask_tensor.cpu().numpy().astype(np.uint8)  # [H, W]

    h, w = mask_np.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)

    for class_idx, color in label2color.items():
        color_mask[mask_np == class_idx] = color

    return color_mask


color_mask = visualize_mask(mask, config.label2color)

plt.imshow(color_mask)
plt.axis("off")
plt.title("Predicted Segmentation Mask")
plt.show()

# Image.fromarray(color_mask).save("predicted_mask.png")